# Single panel model

Introduction:
------------------

Big PV Model (Single panel, flat):  In this model, power output is calculated from a single PV panel per grid cell per time-step. This panel is oriented horizontally (i.e., flat) without considering any shading or slope. The output from this single PV panel is then scaled to the entire neighbourhood and reduced due to effects of geometric placement and shading.

Background:
-----------------
This model was developed using LiDAR and DSM data. Forty simulation areas have been generated in which 25 training areas are used to analyze the relationships between PV power production potential and urban forms.

Later parametrized equations, namely placement efficiency and shading efficiency are generated from the training areas, which were later evaluated using the evaluation areas. Placement efficiency was used to accurately calculate geometrically placed PV panels for an urban area without actually placing PV panels on rooftop building segments. Shading efficiency calculates the PV power output with shading losses instead of using the SEBE model. 

These parametrized equations are used in Big PV model. With the placement and shading efficiency, PV power output for an urban area can be calculated in a simple and fast manner.

Significance:
-----------------
1. With fewer input parameters, can rapidly estimate the PV potential over an entire year.
2. Parametrized equations used in this model assist in calculating geometrically placed PV panels and account for shading losses.

Limitations:
---------------
1. Assumes all rooftop building segments are equipped with PV panels neglecting safety, socio-economic and environmental factors.

Output:
-----------

Hourly PV power output [W/m2] for an urban area

In [32]:
    """

    Input Parameters
    ----------
        l            : Length of PV panel in portait mode [m]
        w            : Width of PV panel in portait mode [m]
        pv_eff       : Efficiency of PV panel [%]
        alpha        : Temperature coefficient of PV panel [1/K]
        lamda_p      : Projected plan area fraction of building segments
        l_s          : Average building segment length [m] is the typical distance of the length of the roof, 
                       which tells you how far the shadows get on the roof and is calculated 
                       by taking the square root of the average segment area.
        f_f          : Fraction of flat building segments
        wire_losses  : Wiring losses
        f_c          : Capacity curve coefficient(f_c = 1 if all the rooftop segments are covered with panels)
        urban_area   : Area of the study area/ urban area [m2]
        metadata     : Input meteorological data specifically formatted using 
                       UMEP -> Pre-processing -> Meteorological data -> Prepare existing data
                       Here, a sample Random Meteorological Day(RMD) for the month of June has been given as an example.
                       Annual weather data for the city of Berlin has been attached for reference.
        sol_angles   : Sun position angles for the urban area.
                       Here, solar angles for the RMD - June has been given as example
        
        Note: 1. In this script, l, w, pv_eff, alpha values has been taken for a standard PV module 
                 but the user can change these parameters according to their module.
              2. lamda_p, l_s, f_f and urban_area values has been taken for an random urban area in the city of Berlin
              3. Input files and parameters can be found under the folder "Datasets" in Github
    """

import numpy as np
import pandas as pd
import math
import pvlib  

l           = 1
w           = 1.6  
pv_eff      = 0.177 
alpha       = -0.0039  
lamda_p     = 0.398  
l_s         = 9.36  
f_f         = 0.413  
wire_losses = 0.98
f_c         = 1
d           = 1.5
β_o         = 53
albedo      = 0.2
metdata     = np.loadtxt("june_RMD.txt", skiprows=1, dtype=float).reshape((-1, 24))
sol_angles  = pd.read_csv(f'june_angles_RMD.csv',index_col=0)

In [33]:
  """
    Constants
    ---------
        ref_temp     : Reference PV cell temperature, default value = 25 °C
        k_t          : Reduction factor, default value = 0.05 °C/(W/m2)
            
            Reduction factor (kT) a constant defined as “expressing the changes in module performance due to 
            differences in the module’s actual , and nominal operating temperatures” (Ramirez Camargo et al., 2015). 
            A constant value for kT = 0.05 °C/(W/m2) was suggested by various authors (King et al., 2004), 
            (Ramirez Camargo et al., 2015) and also used in the PV-GIS web service for 
            building integrated systems.
            
            References:
            ----------
            
            Ramirez Camargo, L., Zink, R., Dorner, W., & Stoeglehner, G. (2015). 
            Spatio-temporal modelling of roof-top photovoltaic panels for improved technical potential assessment 
            and electricity peak load offsetting at the municipal scale. Computers, Environment, and Urban Systems, 
            52, 58–69. https://doi.org/10.1016/j.compenvurbsys.2015.03.002
            
            King, D. L., Boyson, W. E., & Kratochvill, J. A. (2004). Photovoltaic Array Performance Model. December.
            
            
  """

ref_temp = 25
k_t      = 0.05

In [34]:
  """
    Theoretical PV panels
    ---------------------
    
        PV panels are calculated using rooftop building segment area without considering the geometry, tilt, 
        and aspect of the building segment.
        
  """
panel_area = l * w
theoretical_PVP = (f_f * lamda_p / panel_area) + ((1 - f_f) * lamda_p / panel_area)

In [35]:
  """
    Parametrized Equations
    ----------------------
    
        pl_eff       : Placement efficiency is the ratio of geometrically installed PV panels
                       to the theoretical PV panels.
                
        dir_eff      : Direct shading efficiency loss factor is the reduction of direct POA irradiance due to shading 
                       and is estimated by comparing the direct POA irradiance for Models 4 with building resolving model
                       
        dif_eff      : Diffuse shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                       and is estimated by comparing the diffuse POA irradiance for Models 4 with building resolving model
                
  """
    
pl_eff = (0.4435 * f_f + 0.0971)
dir_eff = (-0.00022 * l_s - 0.00420)
dif_eff = (-0.00562 * l_s + 0.43536)

In [36]:
  """
    Geometric PV panels
    -------------------
    
        Geometrical distribution of PV panels on rooftop building segments are calculated 
        using Theoretical PV panels and placement efficiency

  """
geometric_PVP = theoretical_PVP * (1 - pl_eff)

In [37]:
pv_power = pd.DataFrame()

def single_panel(panels):

    for i in range(metdata.shape[0]):

        if metdata[[i], 14] > 0:

            gamma = sol_angles.loc[int(metdata[[i], 2]), 'gamma']
            poa_dir  = metdata[[i], 22] * (1 - (dir_eff * gamma + 1))
            poa_dif  = metdata[[i], 21] * (1 - dif_eff)
            metdata1 = metdata[[i], :]
            hour = int(metdata1[0,2])



            if gamma > 0:
                pv_power.loc[i+1,f'Year']      = int(metdata1[0,0])
                pv_power.loc[i+1,f'Day']       = int(metdata1[0,1])
                pv_power.loc[i+1,f'Hour']      = hour
                pv_power.loc[i+1,f'direct']   = poa_dir * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * poa_dir) - ref_temp)) * panel_area * panels * wire_losses * f_c
                pv_power.loc[i+1,f'diffuse']  = poa_dif * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * poa_dif) - ref_temp)) * panel_area * panels * wire_losses * f_c

    return pv_power

In [38]:
"""
    Returns
    -------
        Theoretical PV power output for an urban area over an given time period.[W/m2]
        
"""
modelled_PVO = single_panel(geometric_PVP)
pv_power['Total_PVP'] = (pv_power['direct'] + pv_power['diffuse']) * urban_area / 1000000

In [39]:
pv_power

,Year,Day,Hour,direct,diffuse,Total_PVP
5,2019.0,167.0,5.0,0.005695,0.674301,0.180335
6,2019.0,171.0,6.0,0.195848,1.445122,0.435185
7,2019.0,160.0,7.0,0.844412,2.434763,0.869637
8,2019.0,152.0,8.0,1.256697,3.353128,1.222526
9,2019.0,175.0,9.0,4.081358,2.875020,1.844832
10,2019.0,179.0,10.0,0.660179,6.411023,1.875283
11,2019.0,169.0,11.0,7.846680,3.500598,3.009298
12,2019.0,174.0,12.0,9.419843,3.528719,3.433959
13,2019.0,156.0,13.0,8.125042,4.384120,3.317430
14,2019.0,165.0,14.0,9.011938,3.377707,3.285734
